## 딥러닝및실습_hw2 - 2020172036 정찬우

해당 작업은 ubuntu 20.04에서 진행되었음

### 1.import 단

In [1]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

### 2.TitanicDatset

In [2]:
class TitanicDataset(Dataset):
  def __init__(self, X, y):
    self.X = torch.FloatTensor(X) #입력데이터
    self.y = torch.LongTensor(y) #타겟 데이터

  def __len__(self):
    return len(self.X) #입력 데이터셋의 크기

  def __getitem__(self, idx): #딕셔너리 형태로 입력과 타겟을 변환
    feature = self.X[idx]
    target = self.y[idx]
    return {'input': feature, 'target': target} 

  def __str__(self): # 데이터셋 샘플 출력
    str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
      len(self.X), self.X.shape, self.y.shape
    )
    return str

입력과 타겟 데이터를 텐서로 변환 및 데이터셋의 크기와 샘플 제공

### 3.titanictestdataset

In [3]:
class TitanicTestDataset(Dataset):
  def __init__(self, X):
    self.X = torch.FloatTensor(X)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    feature = self.X[idx]
    return {'input': feature}

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}".format(
      len(self.X), self.X.shape
    )
    return str

테스트 데이터에 대한 데이터셋 변환, 길이 및 샘플 출력

### 4.전처리_데이터_0

In [4]:
def get_preprocessed_dataset():
    CURRENT_FILE_PATH = os.path.abspath('') #ipynb에서 실행하기 위해 수정함

    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)
    
    # 트레인과 테스트용 데이터 병합
    all_df = pd.concat([train_df, test_df], sort=False) 

    #데이터 전처리 진행
    all_df = get_preprocessed_dataset_1(all_df)

    all_df = get_preprocessed_dataset_2(all_df)

    all_df = get_preprocessed_dataset_3(all_df)

    all_df = get_preprocessed_dataset_4(all_df)

    all_df = get_preprocessed_dataset_5(all_df)

    all_df = get_preprocessed_dataset_6(all_df)

    #전처리된 데이터의 Survived 값이 있는 행을 학습용 데이터로 분리
    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    train_y = train_df["Survived"]

    #Survived값이 비어 있는 행을 테스트용 데이터로 분리 - Survived값 예측
    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    dataset = TitanicDataset(train_X.values, train_y.values)
    #print(dataset)
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2]) # 8:2의 비율로 학습용 검즈용 데이터 분할
    test_dataset = TitanicTestDataset(test_X.values)
    #print(test_dataset)

    return train_dataset, validation_dataset, test_dataset

- CURRENT_FILE_PATH = os.path.dirname(os.path.abspath(__file__))
해당 코드는 ipynb 파일에서 작동하지 않아 수정함

- 데이터 전처리 함수

### 5.전처리_데이터_1

In [5]:
def get_preprocessed_dataset_1(all_df):
    # Pclass별 Fare 평균값을 사용하여 Fare 결측치 메우기
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]

    return all_df

Pclass(객실의 등급)별 Fare(요금의) 결측치(NaN)값을 평균값으로 설정

### 6.전처리_데이터_2

In [6]:
def get_preprocessed_dataset_2(all_df):
    # name을 세 개의 컬럼으로 분리하여 다시 all_df에 합침
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    name_df.columns = ["family_name", "honorific", "name"]
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    all_df = pd.concat([all_df, name_df], axis=1)

    return all_df

탑승자의 Name(이름)을 ,또는 .을 기준으로 성, 지위, 이름으로 분리

### 7. 전처리_데이터_3

In [7]:
def get_preprocessed_dataset_3(all_df):
    # honorific별 Age 평균값을 사용하여 Age 결측치 메우기
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    honorific_age_mean.columns = ["honorific", "honorific_age_mean", ]
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    return all_df

honorific(지위)의 평균값이 아닌 중간값(median)을 이용해 Age(나이)의 결측치 대체함

### 8.전처리_데이터_4

In [8]:
def get_preprocessed_dataset_4(all_df):
    # 가족수(family_num) 컬럼 새롭게 추가
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]

    # 혼자탑승(alone) 컬럼 새롭게 추가
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    all_df["alone"].fillna(0, inplace=True)

    # 학습에 불필요한 컬럼 제거
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    return all_df

기존의 Parch(부모/자녀),SibSp(형재자매 또는 배우자)의 수를 다루는 열을 family_num(가족수)를 다루는 열로 통합함
가족수가 0인 행에 대해서 alone이라는 열에 1을 가족수가 1이상인 행에 대해서 0을 대입함

### 9.전처리_데이터_5

In [9]:
def get_preprocessed_dataset_5(all_df):
    # honorific 값 개수 줄이기
    all_df.loc[
    ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
    ),
    "honorific"
    ] = "other"
    all_df["Embarked"].fillna("missing", inplace=True)

    return all_df

흔하지 않은 경칭들을 전부 other(기타)로 묶어 학습에 미치는 영향을 줄임
Embarked(탑승항구)의 결측값을 missing으로 채움

### 10.전처리_데이터

In [10]:
def get_preprocessed_dataset_6(all_df):
    # 카테고리 변수를 LabelEncoder를 사용하여 수치값으로 변경하기
    category_features = all_df.columns[all_df.dtypes == "object"]
    from sklearn.preprocessing import LabelEncoder
    for category_feature in category_features:
        le = LabelEncoder()
        if all_df[category_feature].dtypes == "object":
          le = le.fit(all_df[category_feature])
          all_df[category_feature] = le.transform(all_df[category_feature])

    return all_df

범주형 범수(카테고리 변수)를 학습시켜 수치형 데이터로 변환

### 11.MyModel

In [11]:
from torch import nn
class MyModel(nn.Module):
  def __init__(self, n_input, n_output): # n_input : n개 특성의 입력데이터, n_output : 예측해야 하는 값(클래스)의 개수 - 이진 문제 2
    super().__init__()

    self.model = nn.Sequential(
      nn.Linear(n_input, 30), #입력을 30개의 노드로 변환
      nn.ReLU(), #ReLU 활성화 함수 사용
      nn.Linear(30, 30), # 은닉층 30개의 노드 -> 30개의 노드로 변환
      nn.ReLU(), #활성화 함수
      nn.Linear(30, n_output), # 출력
    )

  def forward(self, x): #순전파 함수 - 모델을 사용한 값 출력
    x = self.model(x)
    return x

신경망 모델의 정의

### 12.test

In [12]:
def test(test_data_loader):
  print("[TEST]")
  batch = next(iter(test_data_loader)) # 첫 배치 가져옴
  print("{0}".format(batch['input'].shape))
  my_model = MyModel(n_input=11, n_output=2) #입력 특성 11개, 이진 분류
  output_batch = my_model(batch['input'])
  prediction_batch = torch.argmax(output_batch, dim=1) #가장 정확도가 높은 예측 결과를 선택
  for idx, prediction in enumerate(prediction_batch, start=892):
      print(idx, prediction.item()) #892번 인덱스 부터 결과 출력

학습된 모델을 통한 test 결과 출력

### 13.main

In [13]:
if __name__ == "__main__":
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()
    #전처리된 데이터 입력

  print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
    len(train_dataset), len(validation_dataset), len(test_dataset)
  ))
  print("#" * 50, 1)

  for idx, sample in enumerate(train_dataset): #입력 데이터와 타겟 데이터(라벨)의 샘플 출력
    print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

  print("#" * 50, 2)

  train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True) #배치 사이즈 16
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset)) #한번에 데이터 전체 처리

  print("[TRAIN]")
  for idx, batch in enumerate(train_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

  print("[VALIDATION]")
  for idx, batch in enumerate(validation_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

  print("#" * 50, 3)

  test(test_data_loader)

train_dataset: 713, validation_dataset.shape: 178, test_dataset: 418
################################################## 1
0 - tensor([ 1.0000,  0.0000, 56.0000,  0.0000,  1.0000, 83.1583,  0.0000, 87.5090,
         3.0000,  1.0000,  0.0000]): 1
1 - tensor([ 3.0000,  0.0000, 18.0000,  1.0000,  0.0000, 17.8000,  2.0000, 13.3029,
         3.0000,  1.0000,  0.0000]): 0
2 - tensor([  1.0000,   0.0000,  35.0000,   0.0000,   0.0000, 512.3292,   0.0000,
         87.5090,   1.0000,   0.0000,   1.0000]): 1
3 - tensor([ 3.0000,  0.0000, 28.0000,  0.0000,  0.0000,  7.8958,  2.0000, 13.3029,
         1.0000,  0.0000,  1.0000]): 0
4 - tensor([ 3.0000,  0.0000, 22.0000,  0.0000,  0.0000,  7.8792,  1.0000, 13.3029,
         1.0000,  0.0000,  1.0000]): 1
5 - tensor([ 1.0000,  1.0000, 56.0000,  0.0000,  0.0000, 35.5000,  0.0000, 87.5090,
         4.0000,  0.0000,  1.0000]): 1
6 - tensor([ 3.0000,  1.0000, 18.0000,  0.0000,  0.0000,  7.7750,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
7 - te

In [20]:
![kaggle.png][https://drive.google.com/uc?export=<https://drive.google.com/file/d/1YpPV_4tH07cZDypVlYQy1qXmqqewioel>]

/bin/bash: https://drive.google.com/file/d/1YpPV_4tH07cZDypVlYQy1qXmqqewioel: No such file or directory


## 숙제 후기
- 이미지의 경우 학습을 위해 이미지 어그멘테이션과 같은 기법을 활용하여 데이터를 증강한다는 것을 알고 있었다. 이번 과제를 통해 csv와 같은 파일의 경우 결측값 제거, 데이터 병합, LabelEncoder를 활용한 수치화 등의 방식을 활용하여 데이터 증강 및 병합이 된다는 사실을 알게되어 신기했다.
- 마크다운을 이용해 구글드라이브에 업로드한 이미지를 첨부하는데 어려움이 있어 첨부파일로 포함하였다.